# The flu model

<left><img src="util/scooby_doo.gif" width="400px">
    
The flu model is a simple mathematical description of the spread of a flu in a small town. The model divides the population of N individuals into four "compartments" which may vary as a function of time, t:

V(t) are those vulnerable but not yet infected with the flu;

S(t) is the number of sick individuals;

RI(t) are those individuals who either are immune or have recovered from the flu and now have immunity to it.

D(t) are those individuals who deceased.

### Model parameters

The model describes the change in the population of each of these compartments in terms of three parameters, β, γ and α. 

β describes the effective contact rate of the flu: an infected individual comes into contact with βN other individuals per unit time (of which the fraction that are vulnerable to contracting the flu is V/N). 

γ is the mean recovery rate: that is, 1/γ is the mean period of time during which a sick individual can pass it on.

α describes the death rate of the flu: ratio of sick individuals αS that deceased per unit time (of which the fraction that are sick is S/N). 

### Model equations

The differential equations describing are based on the Kermack-McKendrick model [Proc. R. Soc. A, 115, 772 (1927)]:

\begin{equation*}
\frac{dV}{dt}=-\frac{βVS}{N}
\end{equation*}

\begin{equation*}
\frac{dS}{dt}=\frac{βVS}{N}-γS -\frac{αSS}{N}
\end{equation*}

\begin{equation*}
\frac{dR}{dt}=γS
\end{equation*}

\begin{equation*}
\frac{dD}{dt}=\frac{αSS}{N}
\end{equation*}

The following Python code integrates these equations for a flu in a small town of N=1000 people.

#### Import libraries

In [1]:
import numpy as np
import plotly.graph_objs as go
from ipywidgets import widgets

import util.flu_model as flu_model

#### Simulation time range

In [2]:
T = 1000 # days
time_range = np.arange(1,T+1)
population = 100

## Interactive figure
### Function to update the simulation when changing the parameters with the sliders

In [3]:
def update_sim(T, RI_0, S_0, beta, recovery_time, alpha):
    population = flu_model.population(N, RI_0.value, S_0.value, 0)
    D, S, RI, V = flu_model.simulation(time_range, population, beta.value, 1/recovery_time.value, alpha.value)
    return D, S, RI, V

### Function to update the figure when changing the parameters with the sliders

In [4]:
def update_figure(change):
    with fig_flu.batch_animate(duration=0):
        fig_flu.data[0].y = update_sim(T, RI_0, S_0, beta, recovery_time, alpha)[0]
        fig_flu.data[1].y = update_sim(T, RI_0, S_0, beta, recovery_time, alpha)[1]
        fig_flu.data[2].y = update_sim(T, RI_0, S_0, beta, recovery_time, alpha)[2]
        fig_flu.data[3].y = update_sim(T, RI_0, S_0, beta, recovery_time, alpha)[3]

### Definition of the sliders
#### Ratio of population who is initially immune

In [5]:
# Ratio of population who is immune slider
RI_0 = widgets.FloatSlider(min = 0.0,
                        max = 1,
                        value = 0.05, step = 0.01,
                        description = 'Ratio of population who is initially immune: ',
                        continuous_update=True,
                        style = {'description_width': '300px'} ,
                        layout={'width': '700px'})
RI_0.observe(update_figure,names = 'value')

#### Initial number of sick individuals

In [6]:
# Initial number of sick individuals
S_0 = widgets.IntSlider(min = 1,
                        max = 10,
                        value = 5, step = 1,
                        description = 'Initial number of sick individuals: ',
                        continuous_update=True,
                        style = {'description_width': '300px'} ,
                        layout={'width': '700px'})
S_0.observe(update_figure,names = 'value')

#### Contact rate per day (β)

In [7]:
# Beta slider
beta = widgets.FloatSlider(min = 0.0,
                           max = 1,
                           value = 0.20, step = 0.01,
                           description = 'Contact rate per day: ',
                           continuous_update=True,
                           style = {'description_width': '300px'} ,
                           layout={'width': '700px'})
beta.observe(update_figure,names = 'value')

#### Recovery time (1/γ) in days

In [8]:
# Gamma slider
recovery_time = widgets.IntSlider(min = 0,
                           max = 20,
                           value = 15, step = 1,
                           description = 'Recovery time (days): ',
                           continuous_update=True,
                           style = {'description_width': '300px'} ,
                           layout={'width': '700px'})
recovery_time.observe(update_figure,names = 'value')

#### Death rate per day (alpha)

In [9]:
# Alpha slider
alpha = widgets.FloatSlider(min = 0.0,
                           max = 0.1,
                           value = 0.01, step = 0.01,
                           description = 'Death rate per day: ',
                           continuous_update=True,
                           style = {'description_width': '300px'} ,
                           layout={'width': '700px'})
alpha.observe(update_figure,names = 'value')

### Plot the interactive figure
#### Initial simulation

In [10]:
N = 100 # population
population = flu_model.population(N, RI_0.value, S_0.value, 0)
D, S, RI, V = flu_model.simulation(time_range, population, beta.value, 1/recovery_time.value, alpha.value)

#### Plot the data on four separate curves for V(t), S(t), RI(t) and D(t)

In [11]:
D_trace  = go.Scatter(x=time_range, y=D,  name='Deceased', mode='lines',line=dict(width=0, color='black'),stackgroup='one')
S_trace  = go.Scatter(x=time_range, y=S,  name='Sick',  mode='lines',line=dict(width=0, color = 'red'),stackgroup='one')
RI_trace = go.Scatter(x=time_range, y=RI, name='Recovered + Immune', mode='lines',line=dict(width=0, color = 'green'),stackgroup='one')
V_trace  = go.Scatter(x=time_range, y=V,  name='Vulnerable', mode='lines',line=dict(width=0, color = 'blue'),stackgroup='one')

fig_flu  = go.FigureWidget(data   = [D_trace,S_trace,RI_trace,V_trace],
                          layout = go.Layout(xaxis = dict(range = [1,100],title = 'days',rangeslider=dict(visible=True,range=[1,T])),
                                             yaxis = dict(range = [0,N],title = 'population'),
                                             height = 400))

#### Plot

In [12]:
widgets.VBox([widgets.VBox([RI_0, S_0, beta, recovery_time, alpha]),
              fig_flu])